<a href="https://colab.research.google.com/github/ablet-imin/ml-met-trigger/blob/main/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings 
#warnings.filterwarnings('ignore')
#warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from IPython.display import display


In [ ]:
import h5py
with h5py.File('/content/cells_img50_label.h5', 'r') as h5f:
        images  = np.array(h5f['images'])
        labels = np.array(h5f['labels'])
images = images.reshape(-1, images.shape[1]*images.shape[2])
x_train, x_test, y_train, y_test = train_test_split(images, labels,
                                                test_size=0.2, random_state=42,
                                                shuffle=True
                                                )

In [ ]:
#Linear regression
lr_models = {
    'xgr' : XGBRegressor(),
    'lgb' : LGBMRegressor(),
    'lr': LinearRegression(),
    #'rfr': RandomForestRegressor(),
    'kr' : KNeighborsRegressor(),
    #'svr': SVR(kernel='linear'),
    'dtr': DecisionTreeRegressor()
}
nModels = len(lr_models.keys())
scores = pd.DataFrame(index=lr_models.keys(),
                      columns=['train', 'test', 'test_r2'],
                      data=np.zeros((nModels,3)) )
met_pred = {}
def fit_to_data():
  for name, lr in lr_models.items():
    lr.fit(x_train, y_train)
    scores['train'][name] = lr.score(x_train,y_train)
    scores['test'][name] = lr.score(x_test,y_test)
    predicted = lr.predict(x_test)
    scores['test_r2'][name] = r2_score(y_test, predicted,)
    #scores['test_RMS'][name] = mean_squared_error(y_test, predicted, squared=False)
    met_pred[name] = predicted
    print(f'{name:10}OK')

In [ ]:
fit_to_data()

[16:49:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgr       OK
lgb       OK
lr        OK
kr        OK


In [ ]:
#upload results to wandb
!pip install wandb

ls: cannot access '/cells_img50_label.h5': No such file or directory


In [ ]:
import wandb
wandb.login()

In [ ]:
run = wandb.init(project="linearRegression-results")


In [ ]:
wandb.log({
    'scores': scores
})

met_pred['truth']=y_test
wandb.log({
    'prediction': met_pred
})

for name, lr in lr_models.items():
  wandb.log({"name": wandb.plot.pr_curve(y_test, met_pred[name])})

In [ ]:
xs=scores.index()
ys=[scores[sk].to_numpy() for sk in scores.columns()]

In [ ]:
wandb.log({"Score" : wandb.plot.line_series(
          xs=xs,
          ys=ys,
          keys=[sk for sk in scores.columns()],
          title="Two Random Metrics",
          xname="x units")})